# QR分解(经典Gram-Schmidt算法实现)

本文主要介绍QR分解的一种实现算法：经典Gram-Schmidt算法，并且以4x3矩阵$A$为例。

$$
A = [v_1, v_2, v_3] = \left[\begin{array}{ccc}{1} & {0} & {1} \\ {2} & {0} & {0} \\ {0} & {1} & {0} \\ {1} & {-1} & {1}\end{array}\right].
$$

In [7]:
import numpy as np

In [8]:
A = np.array([[1, 0, 1],
              [2, 0, 0],
              [0, 1, 0],
              [1, -1, 1]], dtype=float)
A  # 4 x 3 

array([[ 1.,  0.,  1.],
       [ 2.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 1., -1.,  1.]])

## 经典Gram-Schmidt算法

### 例子

经典Gram-Schmidt算法和Gram-Schmidt正交化相关，基本思想是将还未正交化的向量扣除其在已经正交化的向量上的投影，再对其单位化。以矩阵$A$为例:

**第一步**：$v_1$除上它的长度就得到$q_1$

In [11]:
v1 = A[:, 0]
q1 = v1 / np.linalg.norm(v1)
print("v1:", v1)
print("q1:", q1)
print("length of q1:", np.linalg.norm(q1))

v1: [1. 2. 0. 1.]
q1: [0.40824829 0.81649658 0.         0.40824829]
length of q1: 1.0


**第二步**：$v_2$扣除其在$q_1$上的投影，再单位化得到$q_2$

In [28]:
v2 = A[:, 1]
v2
q2 = (v2 - (q1.T @ v2) * q1)  / np.linalg.norm(v2 - (q1.T @ v2) * q1)
print("v2:", v2)
print("q2:", q2)
print("length of q2:", np.linalg.norm(q2))

v2: [ 0.  0.  1. -1.]
q2: [ 0.12309149  0.24618298  0.73854895 -0.61545745]
length of q2: 0.9999999999999999


**第三步**：$v_3$扣除其在$q_1$、$q_2$上的投影，再单位化得到$q_3$

In [29]:
v3 = A[:, 2]
q3 = (v3 - (q1.T @ v3) * q1 - (q2.T @ v3) * q2) / np.linalg.norm(v3 - (q1.T @ v3) * q1 - (q2.T @ v3) * q2)
print("v3:", v3)
print("q3:", q3)
print("length of q3:", np.linalg.norm(q3))

v3: [1. 0. 0. 1.]
q3: [ 0.69631062 -0.52223297  0.34815531  0.34815531]
length of q3: 0.9999999999999999


这样就得到列正交的$Q$矩阵

In [30]:
Q = np.array([q1, q2, q3]).T
Q # 4 x 3

array([[ 0.40824829,  0.12309149,  0.69631062],
       [ 0.81649658,  0.24618298, -0.52223297],
       [ 0.        ,  0.73854895,  0.34815531],
       [ 0.40824829, -0.61545745,  0.34815531]])

对于上三角矩阵$R$，它的对角线元素为$v$扣除了在前面所有的$q$上的投影之后剩下的长度，而非对角线元素为$v$在前面的各个$q$上的投影量。

In [33]:
r11 = np.linalg.norm(v1)

r12 = q1.T @ v2
r22 = np.linalg.norm(v2 - (q1.T @ v2) * q1)

r13 = q1.T @ v3
r23 = q2.T @ v3
r33 = np.linalg.norm(v3 - (q1.T @ v3) * q1 - (q2.T @ v3) * q2)

R = np.array([
    [r11, r12, r13],
    [  0, r22, r23],
    [  0,   0, r33]
])
R

array([[ 2.44948974, -0.40824829,  0.81649658],
       [ 0.        ,  1.3540064 , -0.49236596],
       [ 0.        ,  0.        ,  1.04446594]])

检验一下$QR$与$A$是否相等：

In [36]:
np.allclose(Q @ R, A)

True

再和`numpy.linalg`的结果比较一下：

In [37]:
np.linalg.qr(A)

(array([[-0.40824829, -0.12309149, -0.69631062],
        [-0.81649658, -0.24618298,  0.52223297],
        [-0.        , -0.73854895, -0.34815531],
        [-0.40824829,  0.61545745, -0.34815531]]),
 array([[-2.44948974,  0.40824829, -0.81649658],
        [ 0.        , -1.3540064 ,  0.49236596],
        [ 0.        ,  0.        , -1.04446594]]))

乍一看好像不一样但不要担心，只是$Q$和$R$都多了一个负号，相乘之后的结果仍然是一样的。

可以证明，对于列满秩的矩阵$A$，如果我们规定$R$的对角线元素为正，则QR分解式是唯一的。

热身结束，来看看代码怎么写。

### Python代码

In [38]:
def CGS(A):
    """
    Args:
        A: (m x n) matrix with n linearly independent columns
    Returns:
        Q: (m x n) matrix with n orthonormal columns
        R: (n x n) upper triangular matrix
    """
    m, n = A.shape
    
    Q = np.zeros(shape=(m, n))
    R = np.zeros(shape=(n, n))
    # 遍历A的列
    for j in range(n):
        vj = A[:, j]
        for i in range(j):
            R[i, j] = Q[:, i].T @ A[:, j]  # 在前面q上的投影量
            vj = vj - R[i, j] * Q[:, i]    # 不断扣除在前面的q上的投影
        R[j, j] = np.linalg.norm(vj)
        Q[:, j] = vj / R[j, j]             # 标准化
        
    return Q, R

In [39]:
CGS(A)

(array([[ 0.40824829,  0.12309149,  0.69631062],
        [ 0.81649658,  0.24618298, -0.52223297],
        [ 0.        ,  0.73854895,  0.34815531],
        [ 0.40824829, -0.61545745,  0.34815531]]),
 array([[ 2.44948974, -0.40824829,  0.81649658],
        [ 0.        ,  1.3540064 , -0.49236596],
        [ 0.        ,  0.        ,  1.04446594]]))

经典Gram-Schmidt算法还是比较直观的，给定向量$v$，不断扣除其在前面所有的$q$上的投影，扣完以后$v$就和它们正交了，只需再单位化即可，通过这种方式一个一个地造标准正交化向量。

而接下来所要谈的改良版Gram-Schmidt算法和它的区别是：给定向量$v$，改良版Gram-Schmidt先标准化得到$q$，然后将后面每一个还未正交化的向量都扣除其在$q$上的投影，这样它们就都和$v$正交了。通过这样的方式，**每一次迭代后面的向量就会和当前的向量正交，而经典Gram-Schmidt算法是当前的向量和之前所有的向量正交**。